In [1]:
from random import choice
from experta import *

In [2]:
class Persoon(Fact):
    naam = Field(str, mandatory=True)
    geslacht = Field(str, mandatory=True)

class Onderwerp(Fact):
    pass

class Relatie(Fact):
    pass

In [43]:
class FamilieBoom(KnowledgeEngine):    
    @DefFacts()
    def init(self):
        yield Persoon(naam="Opa van moeder", geslacht="m", kinderen=["Moeder", "Broer van moeder", "Zus van moeder"])
        yield Persoon(naam="Oma van moeder", geslacht="f", kinderen=["Moeder", "Broer van moeder", "Zus van moeder"])
        yield Persoon(naam="Opa van vader", geslacht="m", kinderen=["Vader", "Broer van vader"])
        yield Persoon(naam="Oma van vader", geslacht="f", kinderen=["Vader", "Broer van vader"])
        
        yield Persoon(naam="Moeder", geslacht="f", kinderen=["Ikzelf", "Broer", "Zus"], ouders=["Opa van moeder", "Oma van moeder"])
        yield Persoon(naam="Vader", geslacht="m", kinderen=["Ikzelf", "Broer", "Zus"], ouders=["Opa van vader", "Oma van vader"])
        
        yield Persoon(naam="Ikzelf", geslacht="m", ouders=["Vader", "Moeder"])
        
        yield Persoon(naam="Broer", geslacht="m", ouders=["Vader", "Moeder"], kinderen=["Zoon van broer"])
        yield Persoon(naam="Zus", geslacht="f", ouders=["Vader", "Moeder"], kinderen=["Dochter van zus"])
        
        yield Persoon(naam="Zoon van broer", geslacht="f", ouders=["Broer"])
        yield Persoon(naam="Dochter van zus", geslacht="m", ouders=["Zus"])
               
    @Rule()
    def invoer(self):
        naam = input("Wat is de naam van het onderwerp (bijv. 'Ikzelf')? ")
        relatie = input("Wat is de relatie die je wilt weten (bijv. 'Ouders')? ")
        self.declare(Onderwerp(naam))
        self.declare(Relatie(relatie))

    @Rule(AS.ow << Onderwerp(MATCH.naam), AS.persoon << Persoon(naam=MATCH.naam), AS.r << Relatie("Grootouders"))
    def GrootOuders(self, ow, r, persoon):
        self.retract(ow)
        self.retract(r)
        self.declare(Onderwerp(persoon["ouders"][0]))
        self.declare(Onderwerp(persoon["ouders"][1]))
        self.declare(Relatie("Ouders"))
        
    @Rule(Onderwerp(MATCH.naam), AS.persoon << Persoon(naam=MATCH.naam), Relatie("Ouders"))
    def Ouders(self, persoon):
        for o in persoon["ouders"]:
            print(o)
            
    @Rule(Onderwerp(MATCH.naam), AS.persoon << Persoon(naam=MATCH.naam, kinderen=W()), Relatie("Kinderen"))
    def Kinderen(self, persoon):
        for k in persoon["kinderen"]:
            print(k)

In [47]:
engine = FamilieBoom()
engine.reset()
engine.run()

Wat is de naam van het onderwerp (bijv. 'Ikzelf')? Vader
Wat is de relatie die je wilt weten (bijv. 'Ouders')? Ouders
Opa van vader
Oma van vader
